#### 什么是文本张量表示

- 将一段文本使用张量进行表示，其中一般将词汇表示成向量，称作词向量，再由各个词向量按顺序组成矩阵形成文本表示

#### e.g.

- ["人生", "该", "如何", "起头"]

- 每个词对应矩阵中的一个向量

- [[1.32, 4.32,  0.32, 5.2],

	[1.32, 4.32,  0.32, 5.2],

	[1.32, 4.32,  0.32, 5.2],

	[1.32, 4.32,  0.32, 5.2]]



#### 文本张量表示的作用

- 将文本表示成张量(矩阵)形式，能够使语言文本可以作为计算机处理程序的输入，进行接下来的一系列的解析工作

#### 文本张量的表示方法

- one-hot编码
- Word2vec
- Word Embedding

#### 什么是one-hot词向量表示

- 又称独热编码，将每个词表示成具有n个元素的向量，这个词向量中只有一个元素是1，其他元素都是0，不同词汇元素为0，不同词汇元素为0的位置不同，其中n的大小整个语料中不同语汇的总数

- e.g.

	- ["人生", "该", "如何", "起头"]

	- [[1.32, 4.32,  0.32, 5.2],

		[1.32, 4.32,  0.32, 5.2],

		[1.32, 4.32,  0.32, 5.2],

		[1.32, 4.32,  0.32, 5.2]]

- one-hot编码实现

In [ ]:
# 导入用于对象保存与加载的joblib
import joblib
# 导入keras中词汇映射器Tokenizer
from keras.preprocessing.text import Tokenizer
# 假定vocab为语料集所有不同词汇集合
vocab = {"周杰伦", "陈奕迅", "王力宏", "李宗盛", "吴一帆", "鹿晗"}
# 实例化一个词汇映射器对象
t = Tokenizer(num_words=None, char_level=False)
# 使用映射器拟合现的的文本数据
t.fit_on_texts(vocab)

for token in vocab:
    zero_list = [0]*len(vocab)
    # 使用映射器转化现有文本数据，每个词汇对应从1开始的自然数
    # 返回样式如 [[2]],取出其中数字需要使用[0][0]
    token_index = t.texts_to_sequences([token])[0][0] - 1
    zero_list[token_index] = 1
    print(token, "的one-hot编码为:", zero_list)

# 使用joblib工具保存映射器，以便之后使用
tokenizer_path = './Tokenizer'
joblib.dump(t, tokenizer_path)

In [ ]:
# 导入用于对象保存与加载的joblib
import joblib
# 加载之前保存的Tokenizer，实例化一个t对象
t = joblib.load(tokenizer_path)

# 编码token为"李宗盛"
token = "李宗盛"
# 使用t获得token_index
token_index = t.texts_to_sequences([token])[0][0] - 1
# 初始化一个zero_list
zero_list = [0] * len(vocab)
# 令zero_list对应索引为1
zero_list[token_index] = 1

print(token, "的one-hot编码为:", zero_list)


#### one-hot编码的优劣：
- 优势：操作简单，容易理解
- 劣势：完全割裂了词与词之间的联系，而且在大语料集下，每个向量的长度过大，占据大量内存

#### 说明

- 正是因为one-hot编码明显的劣势，这种编码方式被应用的地方越来越少，取而代之的是接下来我们要学习的稠密向量表示方法word2vec和word embedding

#### 什么是word2vec:

- 是一种流行的将词汇表示成向量的监督训练方法，该过程将构建神经网络模型，将网络参数作为词汇的向量表示，它包含CBOW和skipgram两种训练模型
	- CBOW(Continuous bag of words)模式
		- 给定一段用于训练的文本语料，再选定某段长度(窗口)作为研究对象。使用上下文词汇预测目标词汇

<img src="/Users/zhangli/Library/Application Support/typora-user-images/image-20230608164755191.png">


- 分析
	- 图中窗口大小 为9，使用前四个词汇对目标词汇进行预测

#### CBOW模式下的word2vec过程说明

- 假设我们给定的训练语料只有一句话:Hope can set you free(愿你自由成长)，窗口大小为3，因此模型的第一个训练样本来自Hope cat set,因为是CBOW模式所以将Hope和set作为输入,cat作为输出，在模型训练时Hope，can,set等词汇都使用它们的one-hot编码，如图所示，每个one-hot编码的单词与各自的变换矩阵(即参数矩阵3*5,这里的3是指最后得到的词向量维度)相乘之后再加，得到上下文表示tdbl(3x1)

	<img src="/Users/zhangli/Library/Application Support/typora-user-images/image-20230608175115068.png">

- 接着，将上下文表示矩阵变换矩阵(参数矩阵5*3.所有的变换矩阵共享参数)相乘，得到5x1的结果矩阵，它将与我们真正的目标矩阵即can的one-hot编码矩阵(5x1)进行损失的计算，然后更新网络参数完成一次模型迭代

<img src="/Users/zhangli/Library/Application Support/typora-user-images/image-20230608175829206.png">

- 最后窗口按序向后移动，重新更新参数，直到所有的语料被遍历完成，得到最终的变换矩阵(3x5),这个变换矩阵与每个词汇的one-hot编码(5x1)相乘，得到 3x1的矩阵就是该词汇word2vec张量表示

#### skipgram模式

- 给定一段用于训练的文本语料，再选定某段长度(窗口)作为研究对象，使用目标词汇预测上下文词汇

	<img src="/Users/zhangli/Library/Application Support/typora-user-images/image-20230608180415276.png">

- 分析

	- 图中窗口大小为9，使用目标词汇对前后四个词汇进行预测

#### skipgram模式下的word2vec过程说明

- 假设我们给定的训练语料只有一句话:Hope can set you free(愿你自由成长)，窗口大小是3，因此模型的第一个训练样本来自Hope can set,因为是skipgram械，所以将使用can作为输入，Hope和set作为输出，在模型训练时，Hope，can,set等词汇使用它们的one-hot编码，如图所示:将can的one-hot编码与变换矩阵(即参数矩阵3x5,这里的3

	指的是最后得到的词向量的维度)相乘，得到目标词汇表示矩阵(3x1)

- 接着，将目标词汇表示矩阵与多个变换矩阵(参数矩阵5x3)相乘，得到多个5x1的结果矩阵，它将与我们Hope和set对应的one-hot编码矩阵(5x1)进行损失的计算，然后更新网络参数完成一次模型迭代

	<img src="/Users/zhangli/Library/Application Support/typora-user-images/image-20230608182826818.png">

- 最后窗口按序向后移动，重新更新参数，直到所有语料被遍历完成，得到最终的变换矩阵即参数矩阵(3x5),这个变换矩阵与每个词汇的one-hot编码(5x1)相乘，得到的3x1的矩阵就是该词汇的word2vec张量表示